# Oracle SODA Python Driver

You can find the documentation for the API [here](https://cx-oracle.readthedocs.io/en/latest/soda.html#)

Import the need python modules

In [66]:
import cx_Oracle
import os

We can now set the location of the directory containing the wallet to enable us to connect to the ATP instance. Once we've done that we can connect to the Oracle ATP instance and get a SODA object to enable us to work with JSON documents.

In [67]:
os.environ['TNS_ADMIN'] = '/home/opc/Wallet'
connection = cx_Oracle.connect('soe', 'ReallyLongPassw0rd', 'sbatp_tpurgent')
soda = connection.getSodaDatabase()

We now need to create JSON collection and if needed add any additional indexes which might accelerate data access.

In [68]:
try:
    collection = soda.createCollection("customers_json")
    collection.createIndex({ "name"   : "customer_index",
                          "fields" : [ { "path"     : "name_last",
                          "datatype" : "string"}]})
except cx_Oracle.DatabaseError as ex:
    print("It looks like the index already exists")

It looks like the index already exists


We can now add data to the collection. Here I'm inserting the document into the database and retrieving it's key. You can find find some examples/test cases on how to use collections [here](https://github.com/oracle/python-cx_Oracle/blob/master/test/SodaCollection.py)

In [69]:
customer_doc = {"id"          : 1,
       "name_last"    : "Giles",
       "name_first"   : "Dom",
       "description"  : "Gold customer, since 1990",
       "account info" : None,
       "dataplan"     : True,
       "phones"       : [{"type" : "mobile", "number" : 9999965499},
                         {"type" : "home",   "number" : 3248723987}]}
doc = collection.insertOneAndGet(customer_doc)
connection.commit()

To fetch documents we could use SQL or Query By Example (QBE) as shown below. You can find further details on QBE [here](). In this example there should just be a single document. 

In [70]:
documents = collection.find().filter({'name_first': {'$eq': 'Dom'}}).getDocuments()
for document in documents:
    data = document.getContent()
    print("{} => {} {}".format(data["id"],data["name_first"],data["name_last"]))

1 => Dom Giles


To update records we can use the `replaceOne` method.

In [71]:
document = collection.find().filter({'name_first': {'$eq': 'Dom'}}).getOne()
updated = collection.find().key(doc.key).replaceOne({"id"          : 1,
       "name_last"    : "Giles",
       "name_first"   : "Dominic",
       "description"  : "Gold customer, since 1990",
       "account info" : None,
       "dataplan"     : True,
       "phones"       : [{"type" : "mobile", "number" : 9999965499},
                         {"type" : "home",   "number" : 3248723987}]},)
connection.commit()

And just to make sure the change happened

In [72]:
data = collection.find().key(document.key).getOne().getContent()
print("{} => {} {}".format(data["id"], data["name_first"], data["name_last"]))

1 => Dominic Giles


And finally we can drop the collection.

In [73]:
try:
    collection.drop()
except cx_Oracle.DatabaseError as ex:
    print("We're were unable to drop the collection")